In [1]:
import os
import pydot
import numpy as np

from pydrake.geometry import StartMeshcat
from pydrake.systems.analysis import Simulator
from IPython.display import HTML, SVG, display
from pydrake.visualization import ModelVisualizer
from pydrake.all import (
  ConstantVectorSource, 
  PiecewisePose,  
  Simulator, 
  StartMeshcat, 
  DiagramBuilder, 
  MeshcatVisualizer, 
  namedview,
  RigidTransform, 
  RollPitchYaw, RotationMatrix, LeafSystem, AbstractValue,
)
from manipulation import running_as_notebook
from manipulation.station import MakeHardwareStation, load_scenario
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.scenarios import AddIiwaDifferentialIK
# from pydrake.systems.System import CreateDefaultContext

from constants import *

In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7006


In [3]:
def make_stack():
    output = ''
    for i in range(NUM_LAYERS):
        # alternate block orientations
        layer_in_x_direction = i % 2 == 0
        for j in range(NUM_BLOCKS_PER_LAYER):
            x = (BLOCK_WIDTH * (j - 1)) if layer_in_x_direction else 0
            y = 0 if layer_in_x_direction else (BLOCK_WIDTH * (j - 1))
            z = BLOCK_HEIGHT * (i + 0.5) + TABLE_HEIGHT
            orientation = "{ deg: [0, 0, 0] }" if layer_in_x_direction else "{ deg: [0, 0, 90] }"
            output += f"""
- add_model:
    name: block_{i * 3 + j}
    file: package://jengabot/models/jenga_block.sdf
    default_free_body_pose:
        jenga_block_link:
            translation: [{x}, {y}, {z}]
            rotation: !Rpy {orientation}"""
    return output

In [12]:
def construct_scenario():
    scenario_data = """
directives:
- add_model:
    name: table_top
    file: package://jengabot/models/table_top.sdf
- add_weld:
    parent: world
    child: table_top::table_top_center
- add_model:
    name: iiwa
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [0]
        iiwa_joint_2: [-2]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-2]
        iiwa_joint_5: [-1.57]
        iiwa_joint_6: [1.57]
        iiwa_joint_7: [0]
- add_weld:
    parent: table_top_link
    child: iiwa::iiwa_link_0
    X_PC:
        translation: [0.35, 0.5, 0.015]
- add_model:
    name: wsg
    file: package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}
"""

    driver_data = """
model_drivers:
    iiwa: !IiwaDriver
      hand_model_name: wsg
    wsg: !SchunkWsgDriver {}
"""

    scenario_data += make_stack()
    scenario_data += driver_data
    return load_scenario(data = scenario_data)

In [13]:
meshcat.Delete()
builder = DiagramBuilder()
scenario = construct_scenario()
station = builder.AddSystem(MakeHardwareStation(scenario, package_xmls=['./package.xml']))
plant = station.GetSubsystemByName("plant")
diagram = builder.Build()
simulator = Simulator(diagram)
context = simulator.get_mutable_context()
station_context = station.GetMyMutableContextFromRoot(context)
plant_context = plant.GetMyMutableContextFromRoot(context)

q0 = plant.GetPositions(plant_context, plant.GetModelInstanceByName("iiwa"))
station.GetInputPort("iiwa.position").FixValue(station_context, q0)

station.GetInputPort("wsg.position").FixValue(station_context, [0])
print(plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName('body')))
simulator.AdvanceTo(0.1)

INFO:drake:Meshcat listening for connections at http://localhost:7016


RigidTransform(
  R=RotationMatrix([
    [0.999999682931835, 0.0007963264582435285, 6.341362301898943e-07],
    [0.000796326710733565, -0.9999993658637701, -0.0007963264582436738],
    [-1.0186502315074795e-17, 0.0007963267107334821, -0.999999682931835],
  ]),
  p=[-0.013128548355729333, 0.32950010812022723, 0.6038851125035964],
)


==== LCM Warning ===
LCM detected that large packets are being received, but the kernel UDP
receive buffer is very small.  The possibility of dropping packets due to
insufficient buffer space is very high.

For more information, visit:
   http://lcm-proj.github.io/lcm/multicast_setup.html

